In [56]:
%matplotlib widget
import caesar
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import scipy 

In [57]:
obj = caesar.load('/Users/ondrea/Downloads/caesar_296.hdf5') # 296=redshift0

yt : [INFO     ] 2021-08-31 14:02:15,730 Opening /Users/ondrea/Downloads/caesar_296.hdf5
yt : [INFO     ] 2021-08-31 14:02:16,000 Found 54135 halos
yt : [INFO     ] 2021-08-31 14:02:16,046 Found 6480 galaxies


In [58]:
#obj.galaxies[1].masses

In [59]:
orig_index = np.arange(0,len(obj.galaxies))
sfr = np.array([gal.sfr for gal in obj.galaxies])
stellar_masses = np.array([gal.masses['stellar'] for gal in obj.galaxies])
bh_masses = np.array([gal.masses['bh'] for gal in obj.galaxies])
gas_masses = np.array([gal.masses['gas'] for gal in obj.galaxies])
metallicity = np.array([gal.metallicities['stellar'] for gal in obj.galaxies])                    
temp = obj.galaxies[1].temperatures['mass_weighted']
radius = np.array([gal.radii['stellar_r20'] for gal in obj.galaxies])


In [60]:
sfr/stellar_masses

array([0., 0., 0., ..., 0., 0., 0.])

In [61]:
bh_masses[bh_masses == 0] = 1e-99
stellar_masses[stellar_masses == 0] = 1e-99
gas_masses[gas_masses == 0] = 1e-99
metallicity[metallicity == 0] = 1e-99

In [62]:
gal_data = pd.DataFrame({'ID': orig_index, 'bh_masses': np.log10(bh_masses),\
                         'stellar_masses': np.log10(stellar_masses), 'sfr': sfr, 'gas_masses' : np.log10(gas_masses),\
                         'metallicity' : np.log10(metallicity), 'radius' : radius})

In [63]:
#What SFRs do we have?
gal_data['sfr'].value_counts()

0.000000    1015
0.195986       1
0.078754       1
0.878105       1
0.034745       1
            ... 
0.158136       1
0.638304       1
1.068277       1
0.933351       1
0.815047       1
Name: sfr, Length: 5466, dtype: int64

In [64]:
#Make a col of specific star formation rate (SFR/gal stellar mass)*(1e9 Gyr/yr)
gal_data["ssfr"] = (sfr/stellar_masses)*1e9 

In [65]:
stellar_masses

array([1.05960459e+12, 8.26104807e+11, 7.95258585e+11, ...,
       2.64359616e+08, 2.61941984e+08, 2.61884624e+08])

In [66]:
gal_data['sfr']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6475    0.0
6476    0.0
6477    0.0
6478    0.0
6479    0.0
Name: sfr, Length: 6480, dtype: float64

In [67]:
gal_data

,ID,bh_masses,stellar_masses,sfr,gas_masses,metallicity,radius,ssfr
0,0,10.290369,12.025144,0.0,-99.0,-1.709673,12.585268,0.0
1,1,9.681071,11.917035,0.0,-99.0,-1.686568,7.108331,0.0
2,2,9.890138,11.900508,0.0,-99.0,-1.713443,8.093110,0.0
3,3,9.811176,11.867450,0.0,-99.0,-1.702289,7.462501,0.0
4,4,9.830744,11.821318,0.0,-99.0,-1.691337,7.025398,0.0
...,...,...,...,...,...,...,...,...
6475,6475,-99.000000,8.431755,0.0,-99.0,-2.948695,2.281374,0.0
6476,6476,-99.000000,8.423798,0.0,-99.0,-2.864811,2.289696,0.0
6477,6477,-99.000000,8.422195,0.0,-99.0,-3.026400,1.733394,0.0
6478,6478,-99.000000,8.418205,0.0,-99.0,-2.740298,2.117305,0.0


In [68]:
pd.plotting.scatter_matrix(gal_data, alpha=0.2, figsize=(10, 10), diagonal="kde");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …